In [ ]:
from pathlib import Path
import numpy as np
import gzip
import matplotlib.pyplot as plt
%matplotlib inline

import sys
if '..' not in sys.path:
    sys.path.append('..')
    
import utils

In [ ]:
def load_pred(path):
    with gzip.open(str(path), 'rb') as f:
        return np.load(f).astype(np.float32) / 1000
    
def show(x, figsize=(10, 8)):
    plt.figure(figsize=figsize)
    plt.imshow(x)

In [ ]:
IM_ID = 200
pred = load_pred('../runs/unet-limit500/{}-pred.npy'.format(IM_ID))
img = utils.load_image(Path('../data/Train/{}.jpg'.format(IM_ID)), cache=False)
show(img)

In [ ]:
for cls in range(utils.N_CLASSES):
    show(pred[cls])

In [ ]:
colored_pred = utils.colored_prediction(pred)
show(colored_pred)

In [ ]:
pred_patch = pred[:, 2000:3000, 4000:]
img_patch = img[2000:3000, 4000:]
colored_patch = colored_pred[2000:3000, 4000:]

In [ ]:
show(4 * colored_patch)

In [ ]:
show(img_patch)

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(img_patch, alpha=0.5)
plt.imshow(3 * colored_patch, alpha=0.5)

In [ ]:
for cls, cls_pred in enumerate(pred_patch):
    print(cls, cls_pred.max())

In [ ]:
def downsample(img, ratio=8):
    h, w = img.shape[:2]
    h = int(h / ratio)
    w = int(w / ratio)
    return cv2.resize(img, (h, w))

show(4 * downsample(colored_patch))

In [ ]:
def show_blobs(bg, blobs, s=4, scale=1):
    bg = bg.copy()
    for y, x, _ in blobs:
        x, y = int(np.round(x * scale)), int(np.round(y * scale))
        bg[y - s: y + s, x - s: x + s] = 1
    show(bg)

In [ ]:
scale = 4
%time cls_blobs_fast = [blob_log(downsample(pred_patch[cls], scale), threshold=0.02, min_sigma=1, max_sigma=4, num_sigma=4) for cls in range(utils.N_CLASSES)]
show_blobs(4 * colored_patch, [b for bs in cls_blobs_fast for b in bs], scale=scale)

In [ ]:
%%time
_ = [blob_log(downsample(pred[cls], 4), threshold=0.02, min_sigma=1, max_sigma=4, num_sigma=4) for cls in range(utils.N_CLASSES)]

In [ ]:
%time cls_blobs = [blob_log(pred_patch[cls], threshold=0.02, min_sigma=4, max_sigma=16, num_sigma=4) for cls in range(utils.N_CLASSES)]
show_blobs(4 * colored_patch, [b for bs in cls_blobs for b in bs])

In [ ]:
import cv2

params = cv2.SimpleBlobDetector_Params()
params.minThreshold = 2;
detector = cv2.SimpleBlobDetector_create(params)

%time cls_keypoints = [detector.detect((pred_patch[cls] * 10000).astype(np.uint8)) for cls in range(utils.N_CLASSES)]

# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
# im_with_keypoints = cv2.drawKeypoints(colored_patch.copy(), keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

In [ ]:
def show_keypoints(bg, keypoints, s=2):
    bg = bg.copy()
    for k in keypoints:
        x, y = k.pt
        x, y = int(np.round(x)), int(np.round(y))
        bg[y - s: y + s, x - s: x + s] = 1
    show(bg)

show_keypoints(4 * colored_patch, [kp for kps in cls_keypoints for kp in kps])